In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [2]:
import ngram
import pickle
from helpers import *


In [3]:
data = pd.read_csv("../../Datasets/dataset_1/train.csv", encoding='utf-8')
data.drop(columns=['id', 'author'], inplace=True)

In [4]:
data.columns = ['Headline',  'articleBody', 'target']
data.head()

,Headline,articleBody,target
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
data.info()
data.columns = ['type', 'articleBody', 'Headline']
data = shuffle(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
id        20800 non-null int64
title     20242 non-null object
author    18843 non-null object
text      20761 non-null object
label     20800 non-null int64
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [5]:
type_rename = {'reliable': 'reliable', 'fake': 'unreliable'}
data['type'] = data['type'].apply(lambda x: type_rename[x])
data.reset_index(drop=True, inplace=True)

In [6]:
data.target.value_counts()

1    10413
0    10387
Name: target, dtype: int64

In [7]:
data.head(10)

,Headline,articleBody,target
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1
5,Jackie Mason: Hollywood Would Love Trump if He...,"In these trying times, Jackie Mason is the Voi...",0
6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,Ever wonder how Britain’s most iconic pop pian...,1
7,Benoît Hamon Wins French Socialist Party’s Pre...,"PARIS — France chose an idealistic, traditi...",0
8,Excerpts From a Draft Script for Donald Trump’...,Donald J. Trump is scheduled to make a highly ...,0
9,"A Back-Channel Plan for Ukraine and Russia, Co...",A week before Michael T. Flynn resigned as nat...,0


In [8]:
# To check for Kaggle dataset
targets = ['0', '1']
targets_dict = dict(zip(targets, ['reliable', 'unreliable']))
targets_dict

{'0': 'reliable', '1': 'unreliable'}

In [ ]:
# For the other dataset
targets = ['0', '1']
targets_dict = dict(zip(targets, ))
targets_dict

In [15]:
data['type'] = list(map(lambda x: targets_dict[str(x)], data['target']))
data.shape

(20800, 4)

In [16]:
data.head(10)

,Headline,articleBody,target,type
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1,unreliable
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0,reliable
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1,unreliable
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1,unreliable
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1,unreliable
5,Jackie Mason: Hollywood Would Love Trump if He...,"In these trying times, Jackie Mason is the Voi...",0,reliable
6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,Ever wonder how Britain’s most iconic pop pian...,1,unreliable
7,Benoît Hamon Wins French Socialist Party’s Pre...,"PARIS — France chose an idealistic, traditi...",0,reliable
8,Excerpts From a Draft Script for Donald Trump’...,Donald J. Trump is scheduled to make a highly ...,0,reliable
9,"A Back-Channel Plan for Ukraine and Russia, Co...",A week before Michael T. Flynn resigned as nat...,0,reliable


In [17]:
data.to_csv('../datasets/kaggle_processed.csv', index=False)

In [3]:
data = pd.read_csv('../datasets/kaggle_processed.csv', encoding='utf-8')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 4 columns):
Headline       20242 non-null object
articleBody    20761 non-null object
target         20800 non-null int64
type           20800 non-null object
dtypes: int64(1), object(3)
memory usage: 650.1+ KB


In [10]:
data['type'].value_counts()

unreliable    10413
reliable      10387
Name: type, dtype: int64

In [5]:
print ("generate unigram")
data["Headline_unigram"] = data["Headline"].astype(str).map(lambda x: preprocess_data(x))
data["articleBody_unigram"] = data["articleBody"].astype(str).map(lambda x: preprocess_data(x))

generate unigram


In [6]:
print ("generate bigram")
join_str = "_"
data["Headline_bigram"] = data["Headline_unigram"].map(lambda x: ngram.getBigram(x, join_str))
data["articleBody_bigram"] = data["articleBody_unigram"].map(lambda x: ngram.getBigram(x, join_str))

generate bigram


In [7]:
print ("generate trigram")
join_str = "_"
data["Headline_trigram"] = data["Headline_unigram"].map(lambda x: ngram.getTrigram(x, join_str))
data["articleBody_trigram"] = data["articleBody_unigram"].map(lambda x: ngram.getTrigram(x, join_str))

generate trigram


In [8]:
with open('../saved_data/kaggle/data_kaggle.pkl', 'wb') as outfile:
    pickle.dump(data, outfile, -1)
    print('dataframe saved in data_kaggle.pkl')

dataframe saved in data_kaggle.pkl


In [9]:
data.head(10)

,Headline,articleBody,target,type,Headline_unigram,articleBody_unigram,Headline_bigram,articleBody_bigram,Headline_trigram,articleBody_trigram
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1,unreliable,"[hous, dem, aid, even, see, comey, letter, jas...","[hous, dem, aid, even, see, comey, letter, jas...","[hous_dem, dem_aid, aid_even, even_see, see_co...","[hous_dem, dem_aid, aid_even, even_see, see_co...","[hous_dem_aid, dem_aid_even, aid_even_see, eve...","[hous_dem_aid, dem_aid_even, aid_even_see, eve..."
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0,reliable,"[flynn, hillari, clinton, big, woman, campus, ...","[ever, get, feel, life, circl, roundabout, rat...","[flynn_hillari, hillari_clinton, clinton_big, ...","[ever_get, get_feel, feel_life, life_circl, ci...","[flynn_hillari_clinton, hillari_clinton_big, c...","[ever_get_feel, get_feel_life, feel_life_circl..."
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1,unreliable,"[whi, truth, might, get, fire]","[whi, truth, might, get, fire, octob, 29, 2016...","[whi_truth, truth_might, might_get, get_fire]","[whi_truth, truth_might, might_get, get_fire, ...","[whi_truth_might, truth_might_get, might_get_f...","[whi_truth_might, truth_might_get, might_get_f..."
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1,unreliable,"[15, civilian, kill, singl, us, airstrik, iden...","[video, 15, civilian, kill, singl, us, airstri...","[15_civilian, civilian_kill, kill_singl, singl...","[video_15, 15_civilian, civilian_kill, kill_si...","[15_civilian_kill, civilian_kill_singl, kill_s...","[video_15_civilian, 15_civilian_kill, civilian..."
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1,unreliable,"[iranian, woman, jail, fiction, unpublish, sto...","[print, iranian, woman, sentenc, six, year, pr...","[iranian_woman, woman_jail, jail_fiction, fict...","[print_iranian, iranian_woman, woman_sentenc, ...","[iranian_woman_jail, woman_jail_fiction, jail_...","[print_iranian_woman, iranian_woman_sentenc, w..."
5,Jackie Mason: Hollywood Would Love Trump if He...,"In these trying times, Jackie Mason is the Voi...",0,reliable,"[jacki, mason, hollywood, would, love, trump, ...","[tri, time, jacki, mason, voic, reason, week, ...","[jacki_mason, mason_hollywood, hollywood_would...","[tri_time, time_jacki, jacki_mason, mason_voic...","[jacki_mason_hollywood, mason_hollywood_would,...","[tri_time_jacki, time_jacki_mason, jacki_mason..."
6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,Ever wonder how Britain’s most iconic pop pian...,1,unreliable,"[life, life, luxuri, elton, john, favorit, sha...","[ever, wonder, britain, icon, pop, pianist, ge...","[life_life, life_luxuri, luxuri_elton, elton_j...","[ever_wonder, wonder_britain, britain_icon, ic...","[life_life_luxuri, life_luxuri_elton, luxuri_e...","[ever_wonder_britain, wonder_britain_icon, bri..."
7,Benoît Hamon Wins French Socialist Party’s Pre...,"PARIS — France chose an idealistic, traditi...",0,reliable,"[benoît, hamon, win, french, socialist, parti,...","[pari, franc, chose, idealist, tradit, candid,...","[benoît_hamon, hamon_win, win_french, french_s...","[pari_franc, franc_chose, chose_idealist, idea...","[benoît_hamon_win, hamon_win_french, win_frenc...","[pari_franc_chose, franc_chose_idealist, chose..."
8,Excerpts From a Draft Script for Donald Trump’...,Donald J. Trump is scheduled to make a highly ...,0,reliable,"[excerpt, draft, script, donald, trump, ampa, ...","[donald, trump, schedul, make, high, anticip, ...","[excerpt_draft, draft_script, script_donald, d...","[donald_trump, trump_schedul, schedul_make, ma...","[excerpt_draft_script, draft_script_donald, sc...","[donald_trump_schedul, trump_schedul_make, sch..."
9,"A Back-Channel Plan for Ukraine and Russia, Co...",A 

## The preprocessing and data pickle Ends Here!
### Next step>>> Get features.

## Generating Targets features for reading labels

In [5]:
with open('../saved_data/kaggle/data_kaggle.pkl', 'rb') as infile:
    data = pickle.load(infile)

In [6]:
data.head(10)

,Headline,articleBody,target,type,Headline_unigram,articleBody_unigram,Headline_bigram,articleBody_bigram,Headline_trigram,articleBody_trigram
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1,unreliable,"[hous, dem, aid, even, see, comey, letter, jas...","[hous, dem, aid, even, see, comey, letter, jas...","[hous_dem, dem_aid, aid_even, even_see, see_co...","[hous_dem, dem_aid, aid_even, even_see, see_co...","[hous_dem_aid, dem_aid_even, aid_even_see, eve...","[hous_dem_aid, dem_aid_even, aid_even_see, eve..."
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0,reliable,"[flynn, hillari, clinton, big, woman, campus, ...","[ever, get, feel, life, circl, roundabout, rat...","[flynn_hillari, hillari_clinton, clinton_big, ...","[ever_get, get_feel, feel_life, life_circl, ci...","[flynn_hillari_clinton, hillari_clinton_big, c...","[ever_get_feel, get_feel_life, feel_life_circl..."
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1,unreliable,"[whi, truth, might, get, fire]","[whi, truth, might, get, fire, octob, 29, 2016...","[whi_truth, truth_might, might_get, get_fire]","[whi_truth, truth_might, might_get, get_fire, ...","[whi_truth_might, truth_might_get, might_get_f...","[whi_truth_might, truth_might_get, might_get_f..."
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1,unreliable,"[15, civilian, kill, singl, us, airstrik, iden...","[video, 15, civilian, kill, singl, us, airstri...","[15_civilian, civilian_kill, kill_singl, singl...","[video_15, 15_civilian, civilian_kill, kill_si...","[15_civilian_kill, civilian_kill_singl, kill_s...","[video_15_civilian, 15_civilian_kill, civilian..."
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1,unreliable,"[iranian, woman, jail, fiction, unpublish, sto...","[print, iranian, woman, sentenc, six, year, pr...","[iranian_woman, woman_jail, jail_fiction, fict...","[print_iranian, iranian_woman, woman_sentenc, ...","[iranian_woman_jail, woman_jail_fiction, jail_...","[print_iranian_woman, iranian_woman_sentenc, w..."
5,Jackie Mason: Hollywood Would Love Trump if He...,"In these trying times, Jackie Mason is the Voi...",0,reliable,"[jacki, mason, hollywood, would, love, trump, ...","[tri, time, jacki, mason, voic, reason, week, ...","[jacki_mason, mason_hollywood, hollywood_would...","[tri_time, time_jacki, jacki_mason, mason_voic...","[jacki_mason_hollywood, mason_hollywood_would,...","[tri_time_jacki, time_jacki_mason, jacki_mason..."
6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,Ever wonder how Britain’s most iconic pop pian...,1,unreliable,"[life, life, luxuri, elton, john, favorit, sha...","[ever, wonder, britain, icon, pop, pianist, ge...","[life_life, life_luxuri, luxuri_elton, elton_j...","[ever_wonder, wonder_britain, britain_icon, ic...","[life_life_luxuri, life_luxuri_elton, luxuri_e...","[ever_wonder_britain, wonder_britain_icon, bri..."
7,Benoît Hamon Wins French Socialist Party’s Pre...,"PARIS — France chose an idealistic, traditi...",0,reliable,"[benoît, hamon, win, french, socialist, parti,...","[pari, franc, chose, idealist, tradit, candid,...","[benoît_hamon, hamon_win, win_french, french_s...","[pari_franc, franc_chose, chose_idealist, idea...","[benoît_hamon_win, hamon_win_french, win_frenc...","[pari_franc_chose, franc_chose_idealist, chose..."
8,Excerpts From a Draft Script for Donald Trump’...,Donald J. Trump is scheduled to make a highly ...,0,reliable,"[excerpt, draft, script, donald, trump, ampa, ...","[donald, trump, schedul, make, high, anticip, ...","[excerpt_draft, draft_script, script_donald, d...","[donald_trump, trump_schedul, schedul_make, ma...","[excerpt_draft_script, draft_script_donald, sc...","[donald_trump_schedul, trump_schedul_make, sch..."
9,"A Back-Channel Plan for Ukraine and Russia, Co...",A 

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 10 columns):
Headline               20242 non-null object
articleBody            20761 non-null object
target                 20800 non-null int64
type                   20800 non-null object
Headline_unigram       20800 non-null object
articleBody_unigram    20800 non-null object
Headline_bigram        20800 non-null object
articleBody_bigram     20800 non-null object
Headline_trigram       20800 non-null object
articleBody_trigram    20800 non-null object
dtypes: int64(1), object(9)
memory usage: 1.6+ MB


In [50]:
processed = data[['type', 'articleBody', 'Headline']]

In [37]:
processed.to_csv('../datasets/processed_kaggle.csv', index_label='id')

In [ ]:
with open('../datasets/processed_k')

In [8]:
targets = data[['Headline', 'target']]

In [9]:
targets.head(10)

,Headline,target
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You Fired,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1
5,Jackie Mason: Hollywood Would Love Trump if He...,0
6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,1
7,Benoît Hamon Wins French Socialist Party’s Pre...,0
8,Excerpts From a Draft Script for Donald Trump’...,0
9,"A Back-Channel Plan for Ukraine and Russia, Co...",0


In [10]:
targets.to_csv('../datasets/targets_kaggle.csv', index_label='id')